In [2]:
# model = Equation (coeff, intercept)

In [ ]:
# save model
# portable ship
# load model
# predict

In [ ]:
train 
test

Hackathon
train+test -> Preprocessing (Hackathon)

train - flight price - carrier - indigo, airindia, jet

testflight price - carrier - indigo, airindia, jet, vistara


combined and onehot -> no problem in forseen

missing value -> imputation -> mean -> train+test


Realworld
Train - Preprocessing



Test - We don't have - Preprocess -> predict


Extra Burden on the devepoler to repeat the step of preprocessing 

Ship the processing code with the model



train - flight price - carrier - indigo, airindia, jet

test - flight price - carrier - indigo, airindia, jet, vistara


separatly did the onehot -> problem 


missing value -> imputation -> mean -> train



In [360]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression

import joblib

In [361]:
data = pd.DataFrame(
{
    'name':['Maruti','honda','maruti','toyOta','hondA','toyota'],
    'fuel_type':['petrol','diesel','petrol','diesel','petrol','diesel'],
    'km_driven' :[2000,4000,100,6344,1234,np.NaN],
    'price' :[200000,400000,424232,800000,500000,900000]
    
}
)
 

In [362]:
data

,name,fuel_type,km_driven,price
0,Maruti,petrol,2000.0,200000
1,honda,diesel,4000.0,400000
2,maruti,petrol,100.0,424232
3,toyOta,diesel,6344.0,800000
4,hondA,petrol,1234.0,500000
5,toyota,diesel,NaN,900000


In [363]:
data['name'].unique()

array(['Maruti', 'honda', 'maruti', 'toyOta', 'hondA', 'toyota'],
      dtype=object)

In [364]:
class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        print('\n>>>>>>>init() called.\n')
    def fit(self, X, y = None):
        print('\n>>>>>>>fit() called.\n')
        return self
    def transform(self, X, y = None):
        print('\n>>>>>>>transform() called.\n')
        print("\n>>>> Input : ",X)
        X_ = X.applymap(lambda x: x.lower())
        print("\n>>>> Output : ",X_)
        print("\n>>>>>>> Custom Transformer Called")
        return X_


In [365]:

preprocess_categorical_encoding = Pipeline([('pre_preprocess', CustomTransformer()),
          ('onehot', OneHotEncoder(sparse=False, handle_unknown='ignore'))
         ])

numerical_encoding_scaling = Pipeline([
                                       ('imputation', SimpleImputer(strategy='mean')),
                                       ('scale', MinMaxScaler())])



>>>>>>>init() called.



In [366]:
preprocess_full = ColumnTransformer(
    transformers=
    [
    ('numerical_prprocessing',numerical_encoding_scaling,['km_driven'],),
    ('categoral_preprocessing',preprocess_categorical_encoding,['name','fuel_type'])
    
    ]
)

In [367]:
pipe = Pipeline([
         ('preprocess',preprocess_full),
         ('model',LinearRegression())])

In [368]:
data.drop(columns = 'price')

,name,fuel_type,km_driven
0,Maruti,petrol,2000.0
1,honda,diesel,4000.0
2,maruti,petrol,100.0
3,toyOta,diesel,6344.0
4,hondA,petrol,1234.0
5,toyota,diesel,NaN


In [369]:
pipe.fit(data.drop(columns = 'price'),y)


>>>>>>>init() called.


>>>>>>>fit() called.


>>>>>>>transform() called.


>>>> Input :       name fuel_type
0  Maruti    petrol
1   honda    diesel
2  maruti    petrol
3  toyOta    diesel
4   hondA    petrol
5  toyota    diesel

>>>> Output :       name fuel_type
0  maruti    petrol
1   honda    diesel
2  maruti    petrol
3  toyota    diesel
4   honda    petrol
5  toyota    diesel

>>>>>>> Custom Transformer Called


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('numerical_prprocessing',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['km_driven']),
                                                 ('categoral_preprocessing',
                                                  Pipeline(steps=[('pre_preprocess',
                                                                   CustomTransformer()),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['name', 'fuel_type'])])),
                ('model', LinearRegression())])

In [370]:
from sklearn import set_config
set_config(display='diagram')  
display(pipe)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('numerical_prprocessing',
                                                  Pipeline(steps=[('imputation',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['km_driven']),
                                                 ('categoral_preprocessing',
                                                  Pipeline(steps=[('pre_preprocess',
                                                                   CustomTransformer()),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['name', 'fuel_type'])])),
                ('model', LinearRegression())])

In [371]:
new_test = ['Toyota','Petrol',np.NaN]

In [372]:
new_ = pd.DataFrame(new_test).T

In [373]:
new_.columns = ['name','fuel_type', 'km_driven']

In [374]:
new_

,name,fuel_type,km_driven
0,Toyota,Petrol,NaN


In [375]:
pipe.predict(new_)


>>>>>>>transform() called.


>>>> Input :       name fuel_type
0  Toyota    Petrol

>>>> Output :       name fuel_type
0  toyota    petrol

>>>>>>> Custom Transformer Called


array([904492.0686354])

In [376]:
joblib.dump(pipe,'preprocess_model.pkl')

['preprocess_model.pkl']